In [11]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB



################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from pandas import read_csv
from searchUtils import findPatternExt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
from spotifyCharts import spotifyCharts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
sd = spotifyData(minYear=1, maxYear=2021)
sd.parse()
sd.saveChartData()

/Volumes/Piggy/Charts/data/spotify/categories
Found 539 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-us-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-us-weekly.p
  --> This file is 1.2MB.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-dk-weekly.p
  --> This file is 364.5kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-dk-weekly.p
  --> This file is 364.5kB.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-gb-weekly.p
  --> This file is 191.4kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-gb-weekly.p
  --> This file is 191.4kB.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-global-weekly.p
  --> This file is 325.5kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-global-weekly.p
  --> This file is 325.5kB.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-in-weekly.p
  

In [30]:
sc = spotifyCharts()
sc.getChartsByRank(2)

  Getting Chart For world
name = world
  Using 1 Charts
  Using 1 Charts


['regional-global-weekly']

In [13]:
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7127 artist keys.
  There are currently 8303 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 127 artist keys.
  There are currently 133 renamed artist keys.


In [33]:
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[0,1])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()

  Getting Chart For us
name = us
  Using 1 Charts
  Using 1 Charts
  Getting Chart For england
name = england
  Using 1 Charts
  Using 1 Charts
  Using Charts (None): ['regional-us-weekly', 'regional-gb-weekly']


Found 9 summary files
Using regional-us-weekly
Using regional-gb-weekly
Renamed 2983 single artists
Renamed 20 multi artists
Saving 838 Artist Album Data to currentSpotifyArtistAlbumData.p
Saving data to currentSpotifyArtistAlbumData.p
  --> This file is 76.7kB.
Saved data to currentSpotifyArtistAlbumData.p
  --> This file is 76.7kB.
Saving 838 Full Artist Data
Saving data to currentSpotifyFullChartArtistAlbumData.p
  --> This file is 118.5kB.
Saved data to currentSpotifyFullChartArtistAlbumData.p
  --> This file is 118.5kB.


In [40]:
sptfy.parse()

/Volumes/Piggy/Charts/data/spotify/categories
Found 209 files.


In [41]:
sptfy.saveChartData()

Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-us-weekly.p
  --> This file is 1.3MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-us-weekly.p
  --> This file is 1.3MB.


In [65]:
sptfy.setFullChartData()

Found 1 summary files
Renamed 0 single artists


In [ ]:
from timeUtils import getDateTime, isDate
from fileUtils import getBaseFilename
from listUtils import isIn
from collections import Counter
from os.path import join
from ioUtils import getFile, saveFile
from searchUtils import findExt

from artistIgnores import getArtistIgnores

class top40Data:
    def __init__(self, minYear=None, maxYear=None, debug=False):
        self.debug  = False
        
        self.basedir  = "/Volumes/Piggy/Charts/"
        self.basename = "Spotify"
                
        self.tc     = top40Charts()
        self.charts = []
        
        self.minYear   = minYear
        self.maxYear   = maxYear
        
        self.dbRenames       = None
        self.multirenameDB   = None

        self.fullChartData   = {}
        self.artistAlbumData = {}
        
        
    def getFullChartDataFilename(self):
        ifile="current{0}FullChartArtistAlbumData.p".format(self.basename)
        return ifile

    def getFullChartData(self):
        return getFile(self.getFullChartDataFilename())
        
    def saveFullChartData(self):
        print("Saving {0} Full Artist Data".format(len(self.fullChartData)))
        saveFile(idata=self.fullChartData, ifile=self.getFullChartDataFilename(), debug=True)        
        
        
    def getArtistAlbumDataFilename(self):
        ifile="current{0}ArtistAlbumData.p".format(self.basename)
        return ifile
    
    def getArtistAlbumData(self):
        return getFile(self.getArtistAlbumDataFilename())
        
    def saveArtistAlbumData(self):
        print("Saving {0} Artist Album Data to {1}".format(len(self.artistAlbumData), self.getArtistAlbumDataFilename()))
        saveFile(idata=self.artistAlbumData, ifile=self.getArtistAlbumDataFilename(), debug=True)
        
    def getArtists(self):
        return list(self.artistAlbumData.keys())
        
        
        
    def setRenames(self, artistRenames):
        self.artistRenames = artistRenames
        
    def setDBRenames(self, dbRenames):
        self.dbRenames = dbRenames
        
    def setMultiDBRenames(self, multirenameDB):
        self.multirenameDB = multirenameDB
        
        
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        
        
    
    def setChartUsage(self, name=None, rank=None):
        if rank is not None:
            if isinstance(rank, list):
                for item in rank:
                    self.charts += self.tc.getChartsByRank(item)
            elif isinstance(rank, int):
                self.charts += self.tc.getChartsByRank(rank)
        elif name is not None:
            self.charts += self.tc.getCharts(name)
        else:
            self.charts = self.tc.getCharts(None)
        if name is None:
            name = "None"
        print("  Using Charts ({0}): {1}".format(name, self.charts))
        
        
    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} files.".format(len(self.files)))
        
        
        
    def setFullChartData(self):   
        dbRenameStats     = 0    
        multiRenameStats  = 0

        fullChartData = {}

        
        self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artistName = item["Artist"]
                    

                    ## Test for rename
                    renamedArtistName = artistName
                    if self.dbRenames is not None:
                        tmpName = self.dbRenames.renamed(renamedArtistName)
                        if tmpName != renamedArtistName:
                            dbRenameStats += 1
                        renamedArtistName = tmpName

                    ## Test for multi rename
                    #renamedArtistName = artistName
                    if self.multirenameDB is not None:
                        tmpName = self.multirenameDB.renamed(renamedArtistName)
                        if tmpName != renamedArtistName:
                            multiRenameStats += 1
                        renamedArtistName = tmpName

                    artist = renamedArtistName

                    renamedArtistName = artist
                    renamedArtistName = renamedArtistName.replace("\r", "").strip()
                    if renamedArtistName != artist:
                        dbRenameStats += 1
                    artist = renamedArtistName

                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                        
                                            
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
                
        print("Renamed {0} single artists".format(dbRenameStats))
        print("Renamed {0} multi artists".format(multiRenameStats))

In [25]:
df = read_csv(ifile, skiprows=1)

In [26]:
df

,Position,Track Name,Artist,Streams,URL
0,1,Bad and Boujee (feat. Lil Uzi Vert),Migos,9212016,https://open.spotify.com/track/4Km5HrUvYTaSUfi...
1,2,Fake Love,Drake,8402709,https://open.spotify.com/track/343YBumqHu19cGo...
2,3,Starboy,The Weeknd,7634586,https://open.spotify.com/track/5aAx2yezTd8zXrk...
3,4,Closer,The Chainsmokers,6384465,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
4,5,Black Beatles,Rae Sremmurd,5745384,https://open.spotify.com/track/6fujklziTHa8uoM...
...,...,...,...,...,...
195,196,She's Mine Pt. 2,J. Cole,978401,https://open.spotify.com/track/5rmcjZTrE9JR1Yr...
196,197,Seven Million (feat. Future),Lil Uzi Vert,976118,https://open.spotify.com/track/6j0teRL3WdExzqm...
197,198,Miss You,James Hersey,973546,https://open.spotify.com/track/7of1slAJIWrXxP8...
198,199,Hello,Adele,970330,https://open.spotify.com/track/4sPmO7WMQUAf45k...
